In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.optimize import minimize 

In [2]:
daily_returns = pd.read_excel("../FERM_3_Assignment_2.xlsx", sheet_name="Daily_Returns_Raw_Data")

In [3]:
daily_returns.head()

,date,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
0,2016-01-04,-0.000736,0.000976,0.014852,-0.018058,0.000000,0.022343,0.008006,-0.023029,-0.010178,...,0.006963,0.026925,0.013954,0.002899,-0.032335,-0.004554,0.020011,0.019501,-0.000378,0.001867
1,2016-01-05,-0.005018,-0.016069,-0.006321,-0.045479,-0.021533,-0.000951,-0.008006,0.018804,-0.005128,...,-0.015441,-0.062643,-0.011614,-0.012273,-0.056744,-0.010708,0.010534,-0.017713,-0.019281,-0.015985
2,2016-01-06,-0.017237,-0.043235,-0.026306,-0.031773,-0.036736,-0.023581,-0.031023,0.074229,-0.012937,...,-0.016280,-0.049700,-0.040530,-0.014546,0.009889,-0.023338,-0.017561,-0.028090,-0.028942,-0.030801
3,2016-01-07,-0.009301,-0.018113,-0.001967,-0.012678,-0.019545,-0.006596,0.000000,-0.018247,-0.007843,...,0.000895,0.037504,0.016888,-0.006605,0.011009,-0.025106,0.001719,-0.025754,-0.016807,-0.017752
4,2016-01-08,0.012082,0.004559,-0.012135,0.018175,0.007211,0.011696,-0.001660,-0.055982,0.002621,...,0.012150,-0.035384,-0.016888,0.014431,0.012092,0.019581,0.020731,-0.010268,0.010637,0.004963


In [4]:
mean_return_matrix = daily_returns.drop(columns=["date"]).mean()
mean_return_matrix

IBM     0.000241
GE     -0.001126
LLY     0.000038
F      -0.000223
BAC     0.001167
KR     -0.000807
ITUB    0.001658
GG      0.000147
SIRI    0.000608
ACB     0.005781
VALE    0.002696
PFE     0.000250
CTL    -0.000815
T       0.000247
ECA     0.001902
ABEV    0.000825
CVS    -0.000569
MRVL    0.001777
CSCO    0.000740
MO      0.000435
BMY    -0.000179
WFC     0.000273
HBAN    0.000614
dtype: float64

In [5]:
cov_matrix = daily_returns.drop(columns=["date"]).cov()
cov_matrix

,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,ACB,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
IBM,0.000127,0.000047,0.000019,0.000055,0.000070,0.000005,0.000073,0.000011,0.000032,0.000031,...,0.000024,0.000084,0.000046,0.000020,0.000058,0.000047,0.000016,0.000014,0.000050,0.000069
GE,0.000047,0.000139,0.000022,0.000070,0.000087,0.000015,0.000084,-0.000026,0.000039,-0.000014,...,0.000039,0.000085,0.000048,0.000039,0.000051,0.000041,0.000020,0.000032,0.000062,0.000076
LLY,0.000019,0.000022,0.000163,0.000030,0.000041,0.000020,0.000030,0.000011,0.000026,0.000011,...,0.000020,0.000019,0.000027,0.000029,0.000025,0.000034,0.000014,0.000071,0.000031,0.000030
F,0.000055,0.000070,0.000030,0.000204,0.000127,0.000035,0.000092,-0.000004,0.000071,0.000026,...,0.000032,0.000152,0.000051,0.000033,0.000089,0.000065,0.000011,0.000055,0.000094,0.000115
BAC,0.000070,0.000087,0.000041,0.000127,0.000296,0.000058,0.000129,-0.000121,0.000085,0.000044,...,0.000019,0.000214,0.000072,0.000045,0.000103,0.000069,-0.000009,0.000040,0.000175,0.000233
KR,0.000005,0.000015,0.000020,0.000035,0.000058,0.000366,-0.000016,-0.000024,0.000030,0.000042,...,0.000015,0.000049,0.000016,0.000063,0.000038,0.000026,0.000009,0.000028,0.000042,0.000043
ITUB,0.000073,0.000084,0.000030,0.000092,0.000129,-0.000016,0.000701,0.000130,0.000070,0.000099,...,0.000052,0.000294,0.000340,0.000038,0.000127,0.000113,0.000050,0.000063,0.000068,0.000106
GG,0.000011,-0.000026,0.000011,-0.000004,-0.000121,-0.000024,0.000130,0.000642,-0.000003,-0.000058,...,0.000030,0.000148,0.000090,-0.000015,0.000009,0.000014,0.000012,0.000016,-0.000079,-0.000110
SIRI,0.000032,0.000039,0.000026,0.000071,0.000085,0.000030,0.000070,-0.000003,0.000169,0.000052,...,0.000029,0.000094,0.000048,0.000020,0.000076,0.000047,0.000019,0.000018,0.000057,0.000063
ACB,0.000031,-0.000014,0.000011,0.000026,0.000044,0.000042,0.000099,-0.000058,0.000052,0.002376,...,-0.000006,0.000149,0.000103,0.000036,0.000066,0.000045,0.000019,-0.000018,0.000025,0.000057


In [6]:
variance = daily_returns.drop(columns=["date"]).var()
variance

IBM     0.000127
GE      0.000139
LLY     0.000163
F       0.000204
BAC     0.000296
KR      0.000366
ITUB    0.000701
GG      0.000642
SIRI    0.000169
ACB     0.002376
VALE    0.001406
PFE     0.000095
CTL     0.000455
T       0.000105
ECA     0.001407
ABEV    0.000313
CVS     0.000193
MRVL    0.000411
CSCO    0.000142
MO      0.000105
BMY     0.000284
WFC     0.000174
HBAN    0.000295
dtype: float64

In [7]:
daily_returns_only = daily_returns.drop(columns=["date"])
daily_returns_only

,IBM,GE,LLY,F,BAC,KR,ITUB,GG,SIRI,ACB,...,T,ECA,ABEV,CVS,MRVL,CSCO,MO,BMY,WFC,HBAN
0,-0.000736,0.000976,0.014852,-0.018058,0.000000,0.022343,0.008006,-0.023029,-0.010178,0.046629,...,0.006963,0.026925,0.013954,0.002899,-0.032335,-0.004554,0.020011,0.019501,-0.000378,0.001867
1,-0.005018,-0.016069,-0.006321,-0.045479,-0.021533,-0.000951,-0.008006,0.018804,-0.005128,-0.006857,...,-0.015441,-0.062643,-0.011614,-0.012273,-0.056744,-0.010708,0.010534,-0.017713,-0.019281,-0.015985
2,-0.017237,-0.043235,-0.026306,-0.031773,-0.036736,-0.023581,-0.031023,0.074229,-0.012937,-0.061485,...,-0.016280,-0.049700,-0.040530,-0.014546,0.009889,-0.023338,-0.017561,-0.028090,-0.028942,-0.030801
3,-0.009301,-0.018113,-0.001967,-0.012678,-0.019545,-0.006596,0.000000,-0.018247,-0.007843,0.026476,...,0.000895,0.037504,0.016888,-0.006605,0.011009,-0.025106,0.001719,-0.025754,-0.016807,-0.017752
4,0.012082,0.004559,-0.012135,0.018175,0.007211,0.011696,-0.001660,-0.055982,0.002621,-0.002378,...,0.012150,-0.035384,-0.016888,0.014431,0.012092,0.019581,0.020731,-0.010268,0.010637,0.004963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,0.006579,0.001716,-0.006328,-0.003967,0.002010,0.025091,-0.006211,0.007981,0.018798,0.030441,...,0.001542,0.004773,0.001596,-0.016616,-0.001816,0.000519,0.002776,-0.008819,-0.000974,-0.004051
498,0.002162,-0.004008,-0.000941,0.001589,-0.003352,-0.016057,0.004662,0.011067,0.009268,0.082888,...,0.003077,0.039678,0.012678,0.003671,-0.007754,-0.001817,-0.000832,0.004909,-0.006847,-0.010201
499,0.001961,-0.002873,0.001763,-0.007968,-0.001680,-0.003243,-0.006221,-0.003150,-0.005550,0.033523,...,-0.004362,0.001524,0.003145,-0.012565,0.009569,0.002077,0.004844,0.006833,-0.002949,-0.002738
500,0.005925,-0.001151,-0.001058,0.006380,0.002352,-0.002168,0.008544,0.002363,-0.001857,0.118306,...,0.007429,0.014367,0.007819,0.003978,-0.007283,0.000778,-0.016145,0.001458,0.005726,0.010228


In [8]:
n_daily = daily_returns_only.shape[1]
n_daily

23

In [9]:
alph = 0.9

## $\mu_1$ for IBM when $\alpha$=0.9

In [42]:
daily_returns_only.mean()["IBM"]

np.float64(0.0002408209122816782)

In [44]:
mu_i = daily_returns_only.mean()["IBM"] * alph + ( (1 - alph) * (np.sum(daily_returns_only.mean()) / n_daily) )
round(mu_i,4)

np.float64(0.0003)

In [11]:
V_est = cov_matrix * alph + ( (1 - alph) * (variance.sum() / n_daily) )

## minimize $x^{T}Vx $ 
## s.t. $\mu^{T}x>=$0.0005
## $\sum_{i=0}^{n} x_i$ = 1

In [12]:
def get_risk(x,cov_matrix):
    x = np.asarray(x, float)
    return float(x.T @ cov_matrix @ x)

def risk_minimizator(mu, cov_matrix, target_return, short_allowed=True):
    mu = np.asarray(mu, float).reshape(-1)
    cov_matrix = np.asarray(cov_matrix, float)
    n = mu.size
    ones = np.ones(n)

    # Initial guess: equal weights
    x0 = np.full(n, 1.0 / n)

    # Constraints:
    # 1) mu^T x - target_return >= 0
    # 2) 1^T x - 1 = 0
    constraints = [
        {"type": "ineq", "fun": lambda x: float(mu @ x) - target_return},
        {"type": "eq",   "fun": lambda x: float(ones @ x) - 1.0},
    ]


    # Bounds (optional): long-only vs allow shorting
    if short_allowed:
        bounds = None
    else:
        bounds = [(0.0, 1.0)] * n

    result = minimize(
        fun=get_risk,
        x0=x0,
        args=(cov_matrix,),
        method="SLSQP",
        constraints=constraints,
        bounds=bounds,
        options={"ftol": 1e-12, "maxiter": 10_000}
)

    if not result.success:
        raise RuntimeError(f"Optimization failed: {result.message}")

    x_star = result.x
    var_star = get_risk(x_star, cov_matrix)
    ret_star = float(mu @ x_star)

    return x_star, var_star, ret_star, result

In [13]:
target_return = 0.0005

In [14]:
w_star, var_star, ret_star, result = risk_minimizator(mean_return_matrix, cov_matrix,
                                                      target_return, short_allowed=False)

In [15]:
w_star

array([7.75032838e-02, 0.00000000e+00, 2.48229100e-02, 0.00000000e+00,
       3.65919034e-02, 1.91643102e-02, 0.00000000e+00, 5.51572314e-02,
       5.31767576e-02, 2.50604238e-02, 0.00000000e+00, 2.00261730e-01,
       0.00000000e+00, 1.27900004e-01, 5.22765226e-19, 3.14685590e-19,
       2.21104417e-02, 3.80531827e-02, 3.65739877e-02, 2.35528313e-01,
       2.82438027e-02, 1.63589576e-02, 3.49276017e-03])

### percent of wealth invested in the stock IBM

In [16]:
round(w_star[0], 3)

np.float64(0.078)

## minimize $x^{T}Vx $ 
## s.t. $\mu^{T}x>=$0.0005
## $\sum_{i=0}^{n} x_i$ = 1
## $\sum_{i=1}^{23} x_{i}^{-}<=$ 0.1
## $ x_{i}^{+} +  x_{i}^{-} =  x_{i} $
## $ x_{i}^{-}>=0, x_{i}^{+}>=0 $

In [22]:
def risk_minimizator(mu, cov_matrix, target_return, max_short, short_allowed=True):
    mu = np.asarray(mu, float).reshape(-1)
    cov_matrix = np.asarray(cov_matrix, float)
    n = mu.size
    ones = np.ones(n)

    if not short_allowed:
    # ---- unchanged long-only branch ----
        x0 = np.full(n, 1.0 / n)

        constraints = [
            {"type": "ineq", "fun": lambda x: float(mu @ x) - target_return},
            {"type": "eq",   "fun": lambda x: float(ones @ x) - 1.0},
            ]

        bounds = [(0.0, 1.0)] * n

        result = minimize(
            fun=get_risk,
            x0=x0,
            args=(cov_matrix,),
            method="SLSQP",
            constraints=constraints,
            bounds=bounds,
            options={"ftol": 1e-12, "maxiter": 10_000}
            )

        if not result.success:
            raise RuntimeError(f"Optimization failed: {result.message}")


        x = result.x
        return {
            "x": x,
            "x_plus": x,
            "x_minus": np.zeros_like(x),
            "variance": get_risk(x, cov_matrix),
            "return": float(mu @ x),
            "result": result,
        }

    # ---- short-allowed with leverage constraint ----
    def split(y):
        y = np.asarray(y, float)
        return y[:n], y[n:]

    def combine(y):
        x_plus, x_minus = split(y)
        return x_plus - x_minus

    def risk_y(y, cov):
        x = combine(y)
        return float(x.T @ cov @ x)

    # Initial guess
    x_plus0 = np.full(n, 1.0 / n)
    x_minus0 = np.zeros(n)
    y0 = np.concatenate([x_plus0, x_minus0])

    constraints = [
        {"type": "ineq", "fun": lambda y: float(mu @ combine(y)) - target_return},
        {"type": "eq",   "fun": lambda y: float(ones @ combine(y)) - 1.0},
        {"type": "ineq", "fun": lambda y: max_short - np.sum(split(y)[1])},
    ]

    bounds = [(0.0, None)] * (2 * n)

    result = minimize(
        fun=risk_y,
        x0=y0,
        args=(cov_matrix,),
        method="SLSQP",
        constraints=constraints,
        bounds=bounds,
        options={"ftol": 1e-12, "maxiter": 10_000}
    )

    if not result.success:
        raise RuntimeError(result.message)

    x_plus, x_minus = split(result.x)
    x = x_plus - x_minus

    return {
        "x": x,
        "x_plus": x_plus,
        "x_minus": x_minus,
        "variance": get_risk(x, cov_matrix),
        "return": float(mu @ x),
        "total_short": float(np.sum(x_minus)),
        "result": result,
    }

In [23]:
max_short = 0.1

In [24]:
res_dic = risk_minimizator(mean_return_matrix, cov_matrix, target_return, max_short, short_allowed=True)

In [25]:
res_dic["x"]

array([ 0.08171208, -0.00713576,  0.02316926, -0.02567339,  0.05704526,
        0.02094783, -0.01418363,  0.06889142,  0.06165255,  0.02259037,
       -0.02200005,  0.18003775, -0.03100717,  0.14103323,  0.00287989,
        0.01080771,  0.0266892 ,  0.03672082,  0.05764956,  0.23306647,
        0.03855286,  0.0252415 ,  0.01131224])

## Compute the percent of wealth invested in the stock IBM

In [29]:
round(res_dic["x"][0], 3)

np.float64(0.082)

In [27]:
res_dic

{'x': array([ 0.08171208, -0.00713576,  0.02316926, -0.02567339,  0.05704526,
         0.02094783, -0.01418363,  0.06889142,  0.06165255,  0.02259037,
        -0.02200005,  0.18003775, -0.03100717,  0.14103323,  0.00287989,
         0.01080771,  0.0266892 ,  0.03672082,  0.05764956,  0.23306647,
         0.03855286,  0.0252415 ,  0.01131224]),
 'x_plus': array([8.17120777e-02, 7.37244382e-19, 2.31692601e-02, 0.00000000e+00,
        5.70452573e-02, 2.09478286e-02, 7.09937144e-19, 6.88914175e-02,
        6.16525494e-02, 2.25903728e-02, 8.46072565e-19, 1.80037753e-01,
        6.95839997e-19, 1.41033234e-01, 2.87989186e-03, 1.08077091e-02,
        2.66892010e-02, 3.67208227e-02, 5.76495575e-02, 2.33066466e-01,
        3.85528638e-02, 2.52415003e-02, 1.13122374e-02]),
 'x_minus': array([0.00000000e+00, 7.13576036e-03, 2.53446771e-19, 2.56733890e-02,
        6.93092996e-20, 2.09553221e-19, 1.41836306e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 2.20000546e-02, 0.00000000e+00,

In [ ]:
risk_minimizator(mean_return_matrix, cov_matrix, target_return, max_short, short_allowed=True)

In [37]:
out1 = risk_minimizator(mean_return_matrix, cov_matrix, 0.0003, max_short, short_allowed=True)
out2 = risk_minimizator(mean_return_matrix, cov_matrix, 0.0010, max_short, short_allowed=True)

x1, x2 = out1["x"], out2["x"]
x3 = 0.5 * x1 + 0.5 * x2

r3 = float(mean_return_matrix @ x3)
v3 = float(x3.T @ cov_matrix @ x3)

out3_star = risk_minimizator(mean_return_matrix, cov_matrix, r3, max_short, short_allowed=True)
x3_star = out3_star["x"]
v3_star = float(x3_star.T @ cov_matrix @ x3_star)

on_frontier = np.isclose(v3, v3_star, rtol=1e-6, atol=1e-5)

print("r3:", r3, "var(x3):", v3, "var(opt at r3):", v3_star, "on_frontier:", on_frontier)


r3: 0.0006499999999984437 var(x3): 3.618800983756848e-05 var(opt at r3): 3.5815478407142245e-05 on_frontier: True
